In [6]:
from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline
from yargy.predicates import gram, eq, custom

def parse_birth_names(content):

    NAME = rule(
        gram('Name'), 
        gram('Surn') 
    )

    name_parser = Parser(NAME)

    names = set()
    for match in name_parser.findall(content):
        name = " ".join(token.value for token in match.tokens)
        names.add(name)

    return names

def parse_birth_locations(content):

    LOCATION = or_(
        rule(eq('родился'), eq('в'), gram('Geox')),  # "родился в Москве"
        rule(eq('родилась'), eq('в'), gram('Geox')),  # "родилась в Санкт-Петербурге"
        rule(eq('родом'), eq('из'), gram('Geox')),  # "родом из Казани"
        rule(eq('дом'), eq('является'), eq('местом'), eq('рождения'), gram('Geox')),  # "дом является местом рождения Владивосток"
        rule(eq('родной'), eq('город'), eq(':'), gram('Geox'))  # "Родной город: Казань"
    )

    location_parser = Parser(LOCATION)

    locations = set()
    for match in location_parser.findall(content):
        location = " ".join(token.value for token in match.tokens)
        locations.add(location)

    return locations


def parse_birth_dates(content):

    def is_integer(value):
        return value.isdigit() and 1 <= len(value) <= 4

    INTEGER = custom(is_integer)

    MONTHS = morph_pipeline([
        'января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
        'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря'
    ])

    BIRTH_PREFIX = morph_pipeline([
        'родился', 'родилась', 'дата рождения'
    ])

    BIRTH_DATE = rule(
        BIRTH_PREFIX,
        INTEGER,
        MONTHS,
        INTEGER.optional(),
        eq('года').optional()
    )

    parser = Parser(BIRTH_DATE)

    dates = set()
    for match in parser.findall(content):
        date = " ".join(token.value for token in match.tokens)
        dates.add(date)
    return dates

def process_file(file_path):

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:

            if not line.strip():
                continue
            
            parts = line.strip().split('\t')
            if len(parts) < 3:
                continue

            rubric, headline, text = parts
            content = f"{headline} {text}"

            names = parse_birth_names(content)
            if not names:
                continue

            locations = parse_birth_locations(content)
            dates = parse_birth_dates(content)

            # 如果日期和地点都为空，跳过
            if not locations and not dates:
                continue

            print(f"Рубрика: {rubric}")
            print(f"Заголовок: {headline}")
            print(f"Имена: {', '.join(names)}")
            print(f"Место рождения: {', '.join(locations) if locations else 'Не найдено'}")
            print(f"Дата рождения: {', '.join(dates) if dates else 'Не найдено'}")
            print()

if __name__ == "__main__":
    file_path = '../data/news.txt'
    process_file(file_path)


Рубрика: culture
Заголовок: Умер звезда сериала «Закон и порядок»
Имена: Арнольдом Шварценеггером
Место рождения: родился в Сиэтле
Дата рождения: Не найдено

Рубрика: forces
Заголовок: Путин назначил руководителя главка МВД по борьбе с коррупцией
Имена: Андрея Курносенко, Дениса Сугробова, Андрей Курносенко, Владимир Путин
Место рождения: родился в Севастополе
Дата рождения: Не найдено

Рубрика: culture
Заголовок: В Артемовске решили демонтировать мемориальную доску Кобзону
Имена: Иосифу Кобзону, Петр Порошенко, Иосиф Кобзон
Место рождения: родился в Артемовске
Дата рождения: Не найдено

Рубрика: culture
Заголовок: Хиросима наградила Йоко Оно за вклад в искусство
Имена: Роберт Раушенберг
Место рождения: родилась в Токио
Дата рождения: Не найдено

Рубрика: culture
Заголовок: В Москве простились с Юрием Яковлевым
Имена: Юрием Яковлевым, Юрия Яковлева, Вера Васильева, Юрий Яковлев, Марианна Вертинская, Андрей Мягков, Василий Лановой, ЯковлевЮрий Яковлев, Людмила Максакова
Место рождения: 